In [2]:
"""
A Cellular automaton simulation of coronavirus cov-19
Created in Apr 2020
parameters need to be modified are highlighted with
##############################
M_INI, C, H, S, TIME_H, TIME_S
M_CONTR, C_CONTR, K_CONTR
##############################
############
time_control
time_total
############
"""


import sys
import random
import numpy as np
import pandas as pd
from numpy import array
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from scipy import signal
# import matplotlib._png as png
from sklearn.cluster import KMeans
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import downscale_local_mean
from skimage.transform import rescale, resize, downscale_local_mean

%matplotlib inline
# np.set_printoptions(threshold=sys.maxsize)

d_rate = 0.2
class Cov():
    """
    this is the model for coronavirus cov-19
    """


    def __init__(self, initial, bi_initial, ini_edge, max_matrix, m, c, m_control, c_control, k):

        """
        modify your parameters over here:
        m, c, h, s, m_control, c_control, k_control...
        """
        ####################################################
        # initial spreading coefficient
        self.M_INI = m
        # self.M_INI = 0.4
        # travel parameter, with area of 15km^2 for one cell
        self.C = c
        # self.C = 0.2
        # percentage to hospital
        self.H = 0.2
        # percentage to selfheal
        self.S = 0.8
        # governmental controls
        self.M_CONTR = m_control
        self.C_CONTR = c_control
        self.K_CONTR = k
        # self.M_CONTR = 0.4
        # self.C_CONTR = 0.2
        # self.K_CONTR = 0
        # time period of latency(days)
        self.TIME_H = 6
        # time period of selfheal(days)
        self.TIME_S = 15
        ####################################################

        """
        initial conditions, no adjustment required
        """
        # initialize time
        self.t = 0
        # maximum population of a cell
        self.MAX = max_matrix
        # initialize the map
        self.state = initial
        self.initial = initial
        self.bi_cmap = bi_initial
        self.edge = ini_edge
        self.m = np.full_like(self.state, self.M_INI)
        # initialize the sum of specific population
        self.redu_h = np.full_like(self.state, 0)
        self.redu_s = np.full_like(self.state, 0)
        self.sum_h = np.full_like(self.state, 0)
        self.sum_s = np.full_like(self.state, 0)
        self.total = np.full_like(self.state, 0)
        # set up neighbor convolution kernel
        """
        convolution kernel used for short-distance migration: 
        assume up, down, left and right have more weights than the other four directions
        0.0 1.0 0.0
        1.0 0.0 1.0
        0.0 1.0 0.0
        """
        self.ker = np.array([[0,1,0],[1,0,1],[0,1,0]])
        # self.ker_sum = np.sum(self.ker)
        self.rows = np.array([], dtype = np.str).reshape(0, len(column_names))
        self.rows_h = np.array([], dtype = np.str).reshape(0, len(column_names))
        self.rows_total = np.array([], dtype = np.str).reshape(0, len(column_names))


    def count_neighb(self):
        """
        use 2D convolution to calculate neighbor weight counts
        """
        neigh_count = signal.convolve2d(self.bi_cmap, self.ker, 'same', boundary = 'fill')
        return neigh_count


    def count_neighb_popu(self, state):
        """
        use 2D convolution to calculate neighbor populations
        """
        popu_count = signal.convolve2d(state, self.ker, 'same', boundary = 'fill')
        return popu_count


    def upadate_m(self):
        """
        update m at each step according to the rules of immunity
        """
        neighb_count = self.count_neighb()+0.001
        # initial_modified = 0
        initial_modified = (self.initial * (1-neighb_count*self.C)**self.t
           + self.count_neighb_popu(self.initial) * self.C * ((1-(1-neighb_count*self.C)**self.t)/(1-(1-neighb_count*self.C))))
        self.total = self.state + self.sum_s + self.sum_h + initial_modified
        self.m = self.M_INI * (1 - self.total/self.MAX)
        # print(self.m[np.where(pi == pi[209][178])])


    def spread(self):
        """
        natural spreading according to the R0 rate and short-distance migration
        """
        neighb = self.count_neighb_popu(self.state)
        num = (self.state + (neighb-self.count_neighb()*self.state)*self.C) * (1 + self.m)
        return num


    def hospital(self):
        """
        population isolated by hospitals
        """
        neighb_count = self.count_neighb()+0.001
        preneighb = self.count_neighb_popu(self.prestate)
        self.redu_h = (self.prestate * self.pre_m * self.H * (1-neighb_count*self.C)**self.TIME_H
            + preneighb * self.pre_m * self.H * self.C * ((1-(1-neighb_count*self.C)**self.TIME_H)/(1-(1-neighb_count*self.C))))
        self.sum_h += self.redu_h
        return self.redu_h


    def selfheal(self):
        """
        population that heal themselves outside of hospitals
        """
        neighb_count = self.count_neighb()+0.001
        preneighb2 = self.count_neighb_popu(self.prestate2)
        self.redu_s = (self.prestate2 * self.pre_m2 * self.S * (1-neighb_count*self.C)**self.TIME_S
            + preneighb2 * self.pre_m2 * self.S * self.C * ((1-(1-neighb_count*self.C)**self.TIME_S)/(1-(1-neighb_count*self.C))))
        self.sum_s += self.redu_s
        return self.redu_s


    def update_early(self):
        """
        early stage, all patients are in the incubation period 
        """
        self.upadate_m()
        self.state = self.spread()
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def update_found(self):
        """
        after a period of 10 days
        first case of being tested positive(hospital isolation) appears
        """
        self.upadate_m()
        self.state = self.spread() - self.hospital()
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def update_selfheal(self):
        """
        after a period of 25 days
        first case of selfheal appears
        """
        self.upadate_m()
        self.state = self.spread() - self.hospital() - self.selfheal()
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def update_control(self):
        """
        introduce governmental control
        """
        self.upadate_m()
        self.m = np.minimum(self.m, np.full_like(self.m, self.M_CONTR))
        self.C = self.C_CONTR
        state_bef_contr = self.spread() - self.hospital() - self.selfheal()
        self.state = state_bef_contr * (1 - self.K_CONTR)
        self.state[self.edge] = 0
        self.state[np.where(self.state < 0)] = 0


    def show_numcases(self):
        """
        show number of cases of every regions
        """
        tmp = 0
        
        totalnum = 0
        totalnum_h = 0
        totalnum_total =0
        
        ans = []
        
        new_row = np.array([])
        new_row_h = np.array([])
        new_row_total = np.array([])
        
        for key, region in regions.items():
            
            numcase = int(self.state[region["loc"]].sum())
            numcase_h = int(self.redu_h[region["loc"]].sum())
            numcase_total = int(self.total[region["loc"]].sum())

            new_row = np.append(new_row, numcase)
            new_row_h = np.append(new_row_h, numcase_h)
            new_row_total = np.append(new_row_total, numcase_total)
            
            totalnum += numcase
            totalnum_h += numcase_h
            totalnum_total += numcase_total
            
            name = region["name"]
            an = plt.annotate("{}:{}".format(name, numcase), xy=(int(500*d_rate), int((25 + tmp*15)*d_rate)))
            ans.append(an)
            tmp += 1
        name = "Total"
        
        numcase = totalnum
        numcase_h = totalnum_h
        numcase_total = totalnum_total
        
        new_row = np.append(new_row, numcase)
        new_row_h = np.append(new_row_h, numcase_h)
        new_row_total = np.append(new_row_total, numcase_total)
        
        an = plt.annotate("{}:{}".format(name, numcase), xy=(int(500*d_rate), int((25 + tmp*15)*d_rate)))
        ans.append(an)
        self.rows = np.vstack((self.rows, new_row))
        self.rows_h = np.vstack((self.rows_h, new_row_h))
        self.rows_total = np.vstack((self.rows_total, new_row_total))
        return ans


    def simulate(self, num_con, num_gen):
        """
        run the simulation by time steps
        """
        fig = plt.figure(figsize = (10,10))
        plt.axis("off")
        states = []
        ms =[]
        ims = []
        cmax = np.log(np.amax(self.MAX)+1)
        # print(np.amax(self.MAX))
        cnorm = mpl.colors.Normalize(vmin=0,vmax=cmax)
        im_edg = plt.imshow(self.bi_cmap, cmap='gray')
        ccmap=mpl.cm.Reds
        ccmap.set_bad(color='aqua')
        
        for i in range(self.TIME_H):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)           
            tm = plt.annotate("DAY {}".format(i+1), xy=(int(25*d_rate),int(25*d_rate)))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_early()

        for i in range(self.TIME_H, self.TIME_S):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)
            tm = plt.annotate("DAY {}".format(i+1), xy=(int(25*d_rate),int(25*d_rate)))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_found()

#         # remove initial numbers
#         neighb_count = self.count_neighb()+0.001
#         initial_modified_ts = (self.initial * (1-neighb_count*self.C)**self.TIME_S
#             + self.count_neighb_popu(self.initial) * self.C * ((1-(1-neighb_count*self.C)**self.TIME_S)/(1-(1-neighb_count*self.C))))
#         self.state = self.state - initial_modified_ts

        for i in range(self.TIME_S, num_con):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            # 25 days before
            self.prestate2 = states[i-self.TIME_S]
            self.pre_m2 = ms[i-self.TIME_S]
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)
            tm = plt.annotate("DAY {}".format(i+1), xy=(int(25*d_rate),int(25*d_rate)))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_selfheal()

        for i in range(num_con, num_gen):
            self.t = i+1
            if np.any(np.log(self.state+1) > cmax):
                break
            if np.any(np.log(self.state+1) < 0):
                print("error: negative numbers occur")
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            # 25 days before
            self.prestate2 = states[i-self.TIME_S]
            self.pre_m2 = ms[i-self.TIME_S]
            im_value = np.log(self.state+1)
            im_value[self.edge] = np.NAN
            # print(len(im_value[np.where(im_value<0)]))
            im = plt.imshow(im_value, cmap=ccmap, norm = cnorm, animated = True)
            tm = plt.annotate("DAY {}".format(i+1), xy=(int(25*d_rate),int(25*d_rate)))
            ims.append([im, tm])
            # ims.append([im, tm] + self.show_numcases())
            self.update_control()
        
        # fig.colorbar(im, extend='both')
        
        # np.savetxt("state40",states[40])
        anim = animation.ArtistAnimation(
            fig, ims , interval=500, repeat=False, blit=True
        )
        plt.close()
        return anim, self.rows, self.rows_h, self.rows_total


"""
this section is for image processing (map of london)
"""


try:
    pi = io.imread('postprocess_ldmap.png')
    # pi = png.read_png_int('postprocess_ukmap.png')
except:
    print("image is not ready, performing image processing...")
    # url = "https://github.com/daweiliucsd/Cov19-model/blob/master/england.png"
    # pi = imread(url)
    pi = png.read_png_int('ldmap.png')
    # pi = io.imread('ldmap.png', 'png')

    print(pi.shape)
    print("white:", pi[0][0])
    # print(pi[0][0].dtype)
    white = pi[0][0]
    colors=[]
    
    print("Bexley:", pi[330][580])
    colors.append(pi[330][580])
    print("City of London:", pi[260][380])
    colors.append(pi[260][380])
    print("Newham:", pi[240][480])
    colors.append(pi[240][480])
    
    print("Croydon:", pi[460][390])
    colors.append(pi[460][390])
    print("Enfield:", pi[70][390])
    colors.append(pi[70][390])
    print("Greenwich:", pi[300][500])
    colors.append(pi[300][500])
    print("Harrow:", pi[130][170])
    colors.append(pi[130][170])
    print("Islington:", pi[200][360])
    colors.append(pi[200][360])
    print("Kingston upon Thames:", pi[420][220])
    colors.append(pi[420][220])
    print("Redbridge:", pi[160][510])
    colors.append(pi[160][510])
    print("Richmond upon Thames:", pi[360][170])
    colors.append(pi[360][170])
    print("Waltham Forest:", pi[150][440])
    colors.append(pi[150][440])
    
    print("Bromley:", pi[450][500])
    colors.append(pi[450][500])
    print("Havering:", pi[200][650])
    colors.append(pi[200][650])
    print("Merton:", pi[400][280])
    colors.append(pi[400][280])
    
    print("Barking and Dagenham:", pi[220][560])
    colors.append(pi[220][560])
    print("Brent:", pi[190][220])
    colors.append(pi[190][220])
    print("Hammersmith and Fulham:", pi[270][260])
    colors.append(pi[270][260])
    print("Haringey:", pi[150][360])
    colors.append(pi[150][360])
    print("Hillingdon:", pi[170][100])
    colors.append(pi[170][100])
    
    print("Hackney:", pi[200][400])
    colors.append(pi[200][400])
    
    print("Ealing:", pi[250][180])
    colors.append(pi[250][180])
    print("Hounslow:", pi[300][150])
    colors.append(pi[300][150])
    print("Lewisham:", pi[350][440])
    colors.append(pi[350][440])
    print("Sutton:", pi[460][300])
    colors.append(pi[460][300])
    
    print("Tower Hamlets:", pi[240][420])
    colors.append(pi[240][420])
    
    print("Lambeth:", pi[330][350])
    colors.append(pi[330][350])
    print("Wandsworth:", pi[340][300])
    colors.append(pi[340][300])
    
    print("Barnet:", pi[100][280])
    colors.append(pi[100][280])
    
    print("Camden:", pi[200][320])
    colors.append(pi[200][320])
    print("Southwark:", pi[300][390])
    colors.append(pi[300][390])
    
    print("Westminster:", pi[250][310])
    colors.append(pi[250][310])
    
    print("Kensington and Chelsea:", pi[270][290])
    colors.append(pi[270][290])
    

    bi_color_pi=np.zeros((pi.shape[0],pi.shape[1]))
    for i, row in enumerate(pi):
        for j, element in enumerate(row):
            if any((element == color).all() for color in colors):
                bi_color_pi[i][j] = 1
            else:
                bi_color_pi[i][j] = 0


    def nearest_nonzero_idx(a,x,y):
        idx = np.argwhere(a)
        idx_i = idx[((idx - [x,y])**2).sum(1).argmin()][0]
        idx_j = idx[((idx - [x,y])**2).sum(1).argmin()][1]
        return idx_i, idx_j


    for i, row in enumerate(pi):
        for j, element in enumerate(row):
            if any((element == color).all() for color in colors):
                continue
            elif (element == white).all():
                continue
            else: 
                # print(nearest_nonzero_idx(bi_color_pi, i, j))
                pi[i][j] = pi[nearest_nonzero_idx(bi_color_pi, i, j)[0]][nearest_nonzero_idx(bi_color_pi, i, j)[1]]


    plt.imshow(pi)
    io.imsave('postprocess_ldmap.png', pi)

    # RGB = pi.reshape(-1, pi.shape[2])
    # print(RGB.shape)
    # for index, cell in enumerate(RGB):
    #     if any((cell == color).all() for color in colors):
    #         continue
    #     else:
    #         RGB[index] = pi[0][0]
    # post_map = RGB.reshape(pi.shape)



    # n_colors = 13
    # arr = pi.reshape(-1, pi.shape[2])
    # kmeans = KMeans(n_clusters=n_colors, random_state=42).fit(arr)
    # labels = kmeans.labels_
    # centers = kmeans.cluster_centers_
    # less_colors = centers[labels].reshape(pi.shape).astype('uint8')

    # io.imshow(less_colors)



"""
initiate situations
online data are saved in a nested-dictionary called regions
pop: population of corresponding region
cases: cov19 cases of corresponding region
loc: index locations of corresponding region
"""


# pi = downscale_local_mean(pi, (10, 10, 1))
pi_3d = io.imread('postprocess_ldmap.png')
# pi_3d = png.read_png_int('postprocess_ukmap.png')
pi = rgb2gray(pi_3d)
pi_d = rescale(pi, d_rate, anti_aliasing=False)
# print(pi.dtype)
# print("image size:", pi.shape)
# print("number of different colors of the original image:", len(np.unique(pi)))
a = pi_d.shape[0]
b = pi_d.shape[1]
cmap = np.zeros((a,b))
cmap_concentrate = np.zeros((a,b))



regions = {
    # Bexley
    "bxl" : {
        "name" : "Bexley",
        "cases" : 0,
        "loc" : np.where(pi_d == pi[330][580])
  },
    # City of London
    "col" : {
        "name" : "City of London",
        "cases" : 0,
        "loc" : np.where(pi_d == pi[260][380])
  },
    # Newham
    "nhm" : {
        "name" : "Newham",
        "cases" : 0,
        "loc" : np.where(pi_d == pi[240][480])
  },
    # Croydon
    "cyd" : {
        "name" : "Croydon",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[460][390])
  },
    # Enfield
    "efd" : {
        "name" : "Enfield",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[70][390])
  },
    # Greenwich
    "grw" : {
        "name" : "Greenwich",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[300][500])
  },
    # Harrow
    "hrw" : {
        "name" : "Harrow",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[130][170])
  },
    # Islington
    "ilt" : {
        "name" : "Islington",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[200][360])
  },
    # Kingston upon Thames
    "kut" : {
        "name" : "Kingston upon Thames",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[420][220])
  },
    # Redbridge
    "rbg" : {
        "name" : "Redbridge",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[160][510])
  },
    # Richmond upon Thames
    "rut" : {
        "name" : "Richmond upon Thames",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[360][170])
  },
    # Waltham Forest
    "whf" : {
        "name" : "Waltham Forest",
        "cases" : 1,
        "loc" : np.where(pi_d == pi[150][440])
  },
    # Bromley
    "bml" : {
        "name" : "Bromley",
        "cases" : 2,
        "loc" : np.where(pi_d == pi[450][500])
  },
    # Havering
    "hvr" : {
        "name" : "Havering",
        "cases" : 2,
        "loc" : np.where(pi_d == pi[200][650])
  },
    # Merton
    "mrt" : {
        "name" : "Merton",
        "cases" : 2,
        "loc" : np.where(pi_d == pi[400][280])
  },
    # Barking and Dagenham
    "bad" : {
        "name" : "Barking and Dagenham",
        "cases" : 3,
        "loc" : np.where(pi_d == pi[220][560])
  },
    # Brent
    "brt" : {
        "name" : "Brent",
        "cases" : 3,
        "loc" : np.where(pi_d == pi[190][220])
  },
    # Hammersmith and Fulham
    "haf" : {
        "name" : "Hammersmith and Fulham",
        "cases" : 3,
        "loc" : np.where(pi_d == pi[270][260])
  },
    # Haringey
    "hrg" : {
        "name" : "Haringey",
        "cases" : 3,
        "loc" : np.where(pi_d == pi[150][360])
  },
    # Hillingdon
    "hld" : {
        "name" : "Hillingdon",
        "cases" : 3,
        "loc" : np.where(pi_d == pi[170][100])
  },
    # Hackney 
    "hkn" : {
        "name" : "Hackney",
        "cases" : 4,
        "loc" : np.where(pi_d == pi[200][400])
  },
    # Ealing
    "eal" : {
        "name" : "Ealing",
        "cases" : 5,
        "loc" : np.where(pi_d == pi[250][180])
  },
    # Hounslow
    "hsl" : {
        "name" : "Hounslow",
        "cases" : 5,
        "loc" : np.where(pi_d == pi[300][150])
  },
    # Lewisham
    "lwh" : {
        "name" : "Lewisham",
        "cases" : 5,
        "loc" : np.where(pi_d == pi[350][440])
  },
    # Sutton
    "sut" : {
        "name" : "Sutton",
        "cases" : 5,
        "loc" : np.where(pi_d == pi[460][300])
  },
    # Tower Hamlets
    "thl" : {
        "name" : "Tower Hamlets",
        "cases" : 6,
        "loc" : np.where(pi_d == pi[240][420])
  }, 
    # Lambeth
    "lmb" : {
        "name" : "Lambeth",
        "cases" : 7,
        "loc" : np.where(pi_d == pi[330][350])
  },
    # Wandsworth
    "wsw" : {
        "name" : "Wandsworth",
        "cases" : 7,
        "loc" : np.where(pi_d == pi[340][300])
  },
    # Barnet
    "bnt" : {
        "name" : "Barnet",
        "cases" : 9,
        "loc" : np.where(pi_d == pi[100][280])
  },
    # Camden
    "cmd" : {
        "name" : "Camden",
        "cases" : 11,
        "loc" : np.where(pi_d == pi[200][320])
  },
    # Southwark
    "swk" : {
        "name" : "Southwark",
        "cases" : 11,
        "loc" : np.where(pi_d == pi[300][390])
  },
    # Westminster
    "wmt" : {
        "name" : "Westminster",
        "cases" : 12,
        "loc" : np.where(pi_d == pi[250][310])
  },
    # Kensington and Chelsea
    "kac" : {
        "name" : "Kensington and Chelsea",
        "cases" : 19,
        "loc" : np.where(pi_d == pi[270][290])
  }    
}


num_total = 0
column_names = []
for key, region in regions.items():
    name = region["name"]
    column_names.append(name)
    num_pi = len(region["loc"][0])
    num_total += num_pi
    random.seed(1)
    idx = random.randint(0, num_pi-1)
    cmap_concentrate[tuple(x[idx] for x in region["loc"])] = region["cases"]
    cmap[region["loc"]] = region["cases"]/num_pi
    # max_matrix[region["loc"]] = region["pop"]/num_pi
column_names.append('All')

bi_cmap = np.zeros((a,b)) + 1
bi_cmap[np.where(pi_d == pi[0][0])] = 0
print("total pixels of land:{}".format(np.sum(bi_cmap)))
max_matrix = np.zeros((a,b)) + 8900000/np.sum(bi_cmap)

# a = pi.shape[0]
# b = pi.shape[1]
# cmap = np.zeros((a,b))
# bi_cmap = np.zeros((a,b))
# for i in range(a):
#     for j in range(b):
#         if pi[i][j][0] == 255 and pi[i][j][1] == 255 and pi[i][j][2] == 255:
#             cmap[i][j] = -1
#         else:
#             cmap[i][j] = pi[i][j][0]/10
#             bi_cmap[i][j] = 1
"""
generate html video
modify simulation time here
time_total: total days of simulation
time_control: the day to add the goverment controls
"""


###################
# time_control should be larger than TIME_S
time_control = 16
time_total = 100

m = 0.4
c = 0.2
m_control = 0.4
c_control = 0.2
k = 0
###################
# print(cmap_concentrate)
init = cmap_concentrate
ocean = np.where(bi_cmap == 0)
Simulation = Cov(init, bi_cmap, ocean, max_matrix, m, c, m_control, c_control, k)
print("Running ......")
anim = Simulation.simulate(time_control, time_total)[0]
# HTML(anim.to_html5_video())


# uncomment the next line if you want to save the video in MP4
anim.save("london.mp4")
# anim.save('animation.gif', writer='imagemagick', fps=10)
print("Finished")


total pixels of land:9456.0
Running ......


/usr/local/lib/python3.6/dist-packages/ipykernel/__main__.py:281: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("Reds"))


Finished


In [1]:
print("Running ......")
for m in [0.23]:
    for c in [0.2]:
        for m_control in [0.3]:
            for c_control in [0.2]:
                for k in [0.11]:
                    Simulation = Cov(init, bi_cmap, ocean, max_matrix, m, c, m_control, c_control, k)
                    ranim, rows, rows_h, rows_total = Simulation.simulate(time_control, time_total)
                    df = pd.DataFrame(rows, columns = column_names)
                    df_h = pd.DataFrame(rows_h, columns = column_names)
                    df_total = pd.DataFrame(rows_total, columns = column_names)
                    # df.to_csv('london_data_tables/m{}_c{}_mc{}_cc{}_k{}.csv'.format(m, c, m_control, c_control, k))
                    df.to_csv('london_daily_hospital_and_total/case_today.csv')
                    df_h.to_csv('london_daily_hospital_and_total/case_today_into_hospital.csv')
                    df_total.to_csv('london_daily_hospital_and_total/case_total.csv')
print("Finished")

Running ......


NameError: name 'Cov' is not defined